# Part 1: Data cleaning from Wiki

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import json
from pandas.io.json import json_normalize


import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'lxml')

# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [3]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [4]:
# create a new DataFrame from the three lists
df = pd.DataFrame({"Postal Code": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
#drop cells with Borough not assigned
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
# group multiple neighborhoods having same postal code
df = df.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
df['Neighborhood'] = df['Neighborhood'].str.replace('/', ',')
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
# rows of dataframe
df.shape

(103, 3)

# Part 2
# Adding Lattitude and Longitude to the dataframe

In [8]:
!pip install geocoder

In [9]:
import geocoder
df1 = pd.read_csv('Geospatial_Coordinates.csv')

In [10]:
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df1.shape

(103, 3)

In [12]:
df= df1.merge(df, on = 'Postal Code')
df.head(10)

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
5,M1J,43.744734,-79.239476,Scarborough,Scarborough Village
6,M1K,43.727929,-79.262029,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,43.711112,-79.284577,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,43.716316,-79.239476,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,43.692657,-79.264848,Scarborough,"Birch Cliff, Cliffside West"


In [13]:
def get_lati_long(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_lati_long('M3A')

[43.75245000000007, -79.32990999999998]

In [14]:
postal_code = df['Postal Code']
coords = [ get_lati_long(postal_code) for postal_code in postal_code.tolist()]

In [15]:
df_coords = pd.DataFrame(coords,columns = ['Latitude','Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [16]:
df.head(15)

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.81139,-79.19662,Scarborough,"Malvern, Rouge"
1,M1C,43.78574,-79.15875,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.76575,-79.17470,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.76812,-79.21761,Scarborough,Woburn
4,M1H,43.76944,-79.23892,Scarborough,Cedarbrae
5,M1J,43.74446,-79.23117,Scarborough,Scarborough Village
6,M1K,43.72582,-79.26461,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,43.71289,-79.28506,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,43.72360,-79.23496,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,43.69510,-79.26466,Scarborough,"Birch Cliff, Cliffside West"


# PART 3:

In [17]:
!pip install geopy

In [18]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [19]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Geographical coordinate of Toranto, Ontario are latitude = {},longitude = {}.'.format(latitude,longitude))

Geographical coordinate of Toranto, Ontario are latitude = 43.6534817,longitude = -79.3839347.


In [20]:
map_toronto = folium.Map(location = [latitude,longitude],zoom_start = 10)
#add Marker
for lat,lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'],df['Neighborhood']):
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(map_toronto)
map_toronto
    

In [21]:
toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop = True)
toronto_data.head()

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M4E,43.67709,-79.29547,East Toronto,The Beaches
1,M4K,43.68375,-79.35512,East Toronto,"The Danforth West, Riverdale"
2,M4L,43.66797,-79.31467,East Toronto,"India Bazaar, The Beaches West"
3,M4M,43.66213,-79.33497,East Toronto,Studio District
4,M4N,43.72843,-79.38713,Central Toronto,Lawrence Park


In [22]:
map_toronto = folium.Map(location = [latitude,longitude],zoom_start = 11)
#add Marker
for lat,lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'],toronto_data['Neighborhood']):
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(map_toronto)
map_toronto

#### Define foursquare credentials and version

In [23]:
CLIENT_ID = '4TDUGDWBR1GHFY45CBKGP3HF000R2CZAV2QOCWMFKBTCJ1KD'
CLIENT_SECRET = 'EXIVFXPHWW2MSYQ4AC3TYH5KAXAZCQ0JBIVUZ2EW0PTXXAIR' 
VERSION = '20210102'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4TDUGDWBR1GHFY45CBKGP3HF000R2CZAV2QOCWMFKBTCJ1KD
CLIENT_SECRET:EXIVFXPHWW2MSYQ4AC3TYH5KAXAZCQ0JBIVUZ2EW0PTXXAIR


##### Exploring Toronto Neighbourhood 

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # results = requests.get(url, "none").json().get('response', {})('groups', [])
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
toronto_venues = getNearbyVenues(names = toronto_data['Neighborhood'], latitudes = toronto_data['Latitude'], longitudes = toronto_data['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [26]:
print('{} venues were returned by Foursquare.'.format(toronto_venues.shape[0]))

1712 venues were returned by Foursquare.


In [27]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,64,64,64,64,64,64
"Brockton, Parkdale Village, Exhibition Place",85,85,85,85,85,85
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",75,75,75,75,75,75
Central Bay Street,63,63,63,63,63,63
Christie,11,11,11,11,11,11
Church and Wellesley,84,84,84,84,84,84
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,30,30,30,30,30,30


In [28]:
print('There are {} uniques categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 227 uniques categories


##### Analyse Each Neighbourhood

In [29]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep ="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']
fixed_columns = [toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [31]:
toronto_grouped.shape
toronto_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.0,0.00,0.015625,0.0,0.015625,0.000000,0.000000,0.000000,0.000000,0.0,0.015625,0.031250,0.000000,0.000000,0.0,0.031250,0.0,0.0,0.0,0.015625,0.0,0.000000,0.0,0.00,0.015625,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.015625,0.015625,0.0,0.000000,0.03125,0.000000,0.000000,0.00,0.0,0.000000,0.015625,0.046875,0.093750,0.0,0.000000,0.0,0.0,0.015625,0.000000,0.015625,0.0,0.000000,0.00,0.015625,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.015625,0.000000,0.0,0.0,0.0,0.0,0.015625,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.03125,0.000000,0.0,0.015625,0.0,0.0,0.000000,0.0,0.00,0.015625,0.015625,0.015625,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.015625,0.015625,0.015625,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,

##### Top 5 Venues

In [32]:
num_top_venues = 5
for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending = False).reset_index(drop=True).head(num_top_venues))
    
    print('\n')


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.05
3         Cheese Shop  0.03
4            Beer Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0          Bar  0.07
1  Coffee Shop  0.06
2         Café  0.06
3   Restaurant  0.05
4    Gift Shop  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
              venue  freq
0       Coffee Shop  0.07
1             Hotel  0.05
2  Asian Restaurant  0.03
3              Café  0.03
4        Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0    Italian Restaurant  0.08
1           Coffee Shop  0.07
2                  Café  0.05
3  Gym / Fitness Center  0.04
4                  Park  0.04


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.11
1  Clo

#### Put in Pandas DataFrame

In [33]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    return row_categories_sorted.index.values[0:num_top_venues]
toronto_grouped.iloc[0,:]

Neighbourhood                               Berczy Park
Accessories Store                                     0
Adult Boutique                                        0
American Restaurant                                   0
Antique Shop                                   0.015625
Aquarium                                              0
Art Gallery                                    0.015625
Art Museum                                            0
Arts & Crafts Store                                   0
Asian Restaurant                                      0
BBQ Joint                                             0
Baby Store                                            0
Bagel Shop                                     0.015625
Bakery                                          0.03125
Bank                                                  0
Bar                                                   0
Basketball Stadium                                    0
Beer Bar                                        

In [34]:
num_top_venues = 10
indicators = ['st' , 'nd' , 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th most common venue'.format(ind+1))

neighbourhood_venues_sorted = pd.DataFrame(columns = columns)
neighbourhood_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhood_venues_sorted.iloc[ind,1:] = return_most_common_venues(toronto_grouped.iloc[ind,:], num_top_venues)

neighbourhood_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Cheese Shop,Restaurant,Beer Bar,Farmers Market,Japanese Restaurant,Shopping Mall
1,"Brockton, Parkdale Village, Exhibition Place",Bar,Coffee Shop,Café,Restaurant,Gift Shop,Sandwich Place,Supermarket,Japanese Restaurant,Furniture / Home Store,French Restaurant
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Asian Restaurant,Café,Restaurant,Steakhouse,Japanese Restaurant,Salad Place,Salon / Barbershop,Sandwich Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Café,Park,Gym / Fitness Center,French Restaurant,Bar,Lounge,Grocery Store,Sandwich Place
4,Central Bay Street,Coffee Shop,Clothing Store,Hotel,Plaza,Middle Eastern Restaurant,Bubble Tea Shop,Cosmetics Shop,Restaurant,Sandwich Place,Café


#### Cluster Neighborhoods

In [35]:
kclusters = 10
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood',1)
kmeans = KMeans(n_clusters = kclusters, init = 'k-means++', max_iter = 100, random_state = 0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 6, 0, 0, 0, 3])

##### creating new dataframe  that include the cluster as well as the top 10 neighborhood


In [36]:
neighbourhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data
toronto_merged = toronto_merged.join(neighbourhood_venues_sorted.set_index('Neighbourhood'), on = 'Neighborhood')

toronto_merged.head()


,Postal Code,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,M4E,43.67709,-79.29547,East Toronto,The Beaches,9.0,Health Food Store,Pub,Neighborhood,Asian Restaurant,Trail,Coffee Shop,Discount Store,Distribution Center,Flea Market,Fish Market
1,M4K,43.68375,-79.35512,East Toronto,"The Danforth West, Riverdale",0.0,Business Service,Coffee Shop,Bus Line,Park,Grocery Store,Discount Store,Ice Cream Shop,Intersection,Fast Food Restaurant,Falafel Restaurant
2,M4L,43.66797,-79.31467,East Toronto,"India Bazaar, The Beaches West",0.0,Sandwich Place,Fast Food Restaurant,Gym,Italian Restaurant,Pet Store,Pub,Dry Cleaner,Restaurant,Movie Theater,Fish & Chips Shop
3,M4M,43.66213,-79.33497,East Toronto,Studio District,0.0,Diner,Pizza Place,Brewery,Italian Restaurant,Bakery,American Restaurant,Arts & Crafts Store,Coffee Shop,Sushi Restaurant,Gastropub
4,M4N,43.72843,-79.38713,Central Toronto,Lawrence Park,2.0,Bus Line,Business Service,Swim School,Yoga Studio,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [37]:

toronto_merged = toronto_merged.dropna(subset = ['Cluster Labels'])
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,Postal Code,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,M4E,43.67709,-79.29547,East Toronto,The Beaches,9,Health Food Store,Pub,Neighborhood,Asian Restaurant,Trail,Coffee Shop,Discount Store,Distribution Center,Flea Market,Fish Market
1,M4K,43.68375,-79.35512,East Toronto,"The Danforth West, Riverdale",0,Business Service,Coffee Shop,Bus Line,Park,Grocery Store,Discount Store,Ice Cream Shop,Intersection,Fast Food Restaurant,Falafel Restaurant
2,M4L,43.66797,-79.31467,East Toronto,"India Bazaar, The Beaches West",0,Sandwich Place,Fast Food Restaurant,Gym,Italian Restaurant,Pet Store,Pub,Dry Cleaner,Restaurant,Movie Theater,Fish & Chips Shop
3,M4M,43.66213,-79.33497,East Toronto,Studio District,0,Diner,Pizza Place,Brewery,Italian Restaurant,Bakery,American Restaurant,Arts & Crafts Store,Coffee Shop,Sushi Restaurant,Gastropub
4,M4N,43.72843,-79.38713,Central Toronto,Lawrence Park,2,Bus Line,Business Service,Swim School,Yoga Studio,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [39]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)
x = np.arange(kclusters)
ys = [i + x + (1*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# THANK YOU!....